<a href="https://colab.research.google.com/github/jmins33/ml_edu/blob/master/Financial_Industry_A/%ED%98%84%EB%8C%80%EB%B0%B1%ED%99%94%EC%A0%90_%EC%A3%BC%EA%B0%80_%EC%9D%BC%EB%B3%84%EC%8B%9C%EC%84%B8_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 현대백화점 주가 일별 시세 크롤링

참고 : http://blog.quantylab.com/crawling_naverfin_daycandle.html

In [ ]:
code = '069960'

In [ ]:
import requests
url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)
res = requests.get(url)
res.encoding = 'utf-8'
res.status_code

200

In [ ]:
url

'http://finance.naver.com/item/sise_day.nhn?code=069960'

In [ ]:
from bs4 import BeautifulSoup
soap = BeautifulSoup(res.text, 'lxml')

In [ ]:
el_table_navi = soap.find("table", class_="Nnavi")
el_td_last = el_table_navi.find("td", class_="pgRR")
pg_last = el_td_last.a.get('href').rsplit('&')[1]
pg_last = pg_last.split('=')[1]
pg_last = int(pg_last)
pg_last

441

In [ ]:
import traceback
import pandas as pd

def parse_page(code, page):
    try:
        url = 'http://finance.naver.com/item/sise_day.nhn?code={code}&page={page}'.format(code=code, page=page)
        res = requests.get(url)
        _soap = BeautifulSoup(res.text, 'lxml')
        _df = pd.read_html(str(_soap.find("table")), header=0)[0]
        _df = _df.dropna()
        return _df
    except Exception as e:
        traceback.print_exc()
    return None

In [ ]:
parse_page(code, 1)

,날짜,종가,전일비,시가,고가,저가,거래량
1,2020.09.15,57800.0,600.0,57500.0,58000.0,56600.0,200761.0
2,2020.09.14,57200.0,900.0,57000.0,58400.0,56700.0,324102.0
3,2020.09.11,56300.0,100.0,56400.0,57400.0,56200.0,269437.0
4,2020.09.10,56400.0,400.0,56500.0,57200.0,55900.0,262036.0
5,2020.09.09,56000.0,400.0,55900.0,56600.0,55400.0,125147.0
9,2020.09.08,56400.0,100.0,56800.0,57100.0,56300.0,152605.0
10,2020.09.07,56300.0,500.0,56100.0,57100.0,55600.0,174472.0
11,2020.09.04,55800.0,700.0,55300.0,56200.0,55100.0,194702.0
12,2020.09.03,56500.0,0.0,57000.0,58000.0,56500.0,287154.0
13,2020.09.02,56500.0,1600.0,56100.0,57800.0,55300.0,370906.0


In [ ]:
parse_page(code, 2)

,날짜,종가,전일비,시가,고가,저가,거래량
1,2020.09.01,54900.0,1200.0,54300.0,55400.0,53800.0,320255.0
2,2020.08.31,53700.0,2000.0,56000.0,57200.0,53700.0,1731829.0
3,2020.08.28,55700.0,1100.0,55000.0,56700.0,54900.0,261233.0
4,2020.08.27,54600.0,2000.0,56300.0,56300.0,54400.0,319492.0
5,2020.08.26,56600.0,800.0,57100.0,57200.0,56100.0,179178.0
9,2020.08.25,57400.0,900.0,56900.0,58100.0,56800.0,170652.0
10,2020.08.24,56500.0,1200.0,57500.0,57800.0,56000.0,157000.0
11,2020.08.21,57700.0,600.0,58300.0,59200.0,57400.0,120640.0
12,2020.08.20,58300.0,2000.0,59900.0,60400.0,57900.0,155463.0
13,2020.08.19,60300.0,400.0,60200.0,60900.0,59500.0,90116.0


In [ ]:
import datetime
str_datefrom = datetime.datetime.strftime(datetime.datetime(year=2020, month=1, day=1), '%Y.%m.%d')
str_datefrom

'2020.01.01'

In [ ]:
str_dateto = datetime.datetime.strftime(datetime.datetime.today(), '%Y.%m.%d')
str_dateto

'2020.09.15'

In [ ]:
#!pip install html5lib

In [ ]:
df = None
for page in range(1, pg_last+1):
    _df = parse_page(code, page)
    _df_filtered = _df[_df['날짜'] > str_datefrom]
    if df is None:
        df = _df_filtered
    else:
        df = pd.concat([df, _df_filtered])
    if len(_df) > len(_df_filtered):
        break

In [ ]:
df = df.reset_index(drop=True)
df

,날짜,종가,전일비,시가,고가,저가,거래량
0,2020.09.15,57700.0,500.0,57500.0,58000.0,56600.0,204615.0
1,2020.09.14,57200.0,900.0,57000.0,58400.0,56700.0,324102.0
2,2020.09.11,56300.0,100.0,56400.0,57400.0,56200.0,269437.0
3,2020.09.10,56400.0,400.0,56500.0,57200.0,55900.0,262036.0
4,2020.09.09,56000.0,400.0,55900.0,56600.0,55400.0,125147.0
...,...,...,...,...,...,...,...
172,2020.01.08,87300.0,1600.0,87800.0,89400.0,86500.0,105155.0
173,2020.01.07,88900.0,4900.0,84300.0,89900.0,84100.0,183531.0
174,2020.01.06,84000.0,1600.0,84400.0,85600.0,83300.0,68657.0
175,2020.01.03,85600.0,300.0,85900.0,86500.0,83900.0,95600.0


In [ ]:
df.drop(['전일비'], axis='columns', inplace=True)
df.head()

,날짜,종가,시가,고가,저가,거래량
0,2020.09.15,57700.0,57500.0,58000.0,56600.0,204615.0
1,2020.09.14,57200.0,57000.0,58400.0,56700.0,324102.0
2,2020.09.11,56300.0,56400.0,57400.0,56200.0,269437.0
3,2020.09.10,56400.0,56500.0,57200.0,55900.0,262036.0
4,2020.09.09,56000.0,55900.0,56600.0,55400.0,125147.0


In [ ]:
df = df[['날짜', '시가', '고가', '저가', '종가', '거래량']]
df.head()

,날짜,시가,고가,저가,종가,거래량
0,2020.09.15,57500.0,58000.0,56600.0,57700.0,204615.0
1,2020.09.14,57000.0,58400.0,56700.0,57200.0,324102.0
2,2020.09.11,56400.0,57400.0,56200.0,56300.0,269437.0
3,2020.09.10,56500.0,57200.0,55900.0,56400.0,262036.0
4,2020.09.09,55900.0,56600.0,55400.0,56000.0,125147.0


In [ ]:
# csv 파일 저장
#import os
#path_dir = 'data/2018-07-05-crawling'
#if not os.path.exists(path_dir):
#    os.makedirs(path_dir)
#path = os.path.join(path_dir, '{code}_{date_from}_{date_to}.csv'.format(code=code, date_from=str_datefrom, date_to=str_dateto))
#path

In [ ]:
# csv 파일 저장
df.to_csv('C:/Users/jminw/Untitled Folder 2/hyundaistock.csv', index=False)

### SQLITE 연결 및 데이터 가져오기

In [ ]:
import sqlite3

In [ ]:
dbname='hyundaiStock.db'
with sqlite3.connect(dbname) as conn:
    cur=conn.cursor()
    sql='select * from StockDay'
    result = cur.execute(sql)
    rows=cur.fetchall()
    cur.close()
rows

[(1, '069960', '현대백화점', 20200900, 100, 200, 90, 150, 22000),
 (2, '069960', '현대백화점', 20200901, 50, '', '', '', '')]

In [ ]:
with sqlite3.connect(dbname) as conn:
    cur=conn.cursor()
    sql='insert into StockDay \
    (SIDX, SCODE, SCOMP, SDAY, OPENP, HIGHP, LOWP,CLOSEP,VOL) \
    values (?,?,?,?,?,?,?,?,?)'
    data=[1001,60000,'신세계','2000830',30,60,70,50,250]
    #실행하기
    cur.execute(sql,data)
    # 변경사항 저장하기
    conn.commit()
    cur.close()

In [ ]:
dbname='hyundaiStock.db'
with sqlite3.connect(dbname) as conn:
    cur=conn.cursor()
    sql='select * from StockDay'
    result = cur.execute(sql)
    rows=cur.fetchall()
    cur.close()
rows

[(1, '069960', '현대백화점', 20200900, 100, 200, 90, 150, 22000),
 (2, '069960', '현대백화점', 20200901, 50, '', '', '', ''),
 (1001, '60000', '신세계', 2000830, 30, 60, 70, 50, 250)]

In [ ]:
def addData(data)
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        sql='insert into StockDay \
        (SIDX, SCODE, SCOMP, SDAY, OPENP, HIGHP, LOWP,CLOSEP,VOL) \
        values (?,?,?,?,?,?,?,?,?)'
        #data=[1001,60000,'신세계','2000830',30,60,70,50,250]
        #실행하기
        cur.execute(sql,data)
        # 변경사항 저장하기
        conn.commit()
        cur.close()